In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

In [2]:
from fbref import BotScrap

## Here I will trying to filter data from FBREF

## Game 

In [3]:
data = BotScrap()

In [4]:
match_stats = data.get_match('https://fbref.com/en/matches/abf3df21/South-Africa-Italy-August-2-2023-FIFA-Womens-World-Cup')

In [24]:
match_stats['shots']

,0
0,Minute Player Squ...
1,Minute Player Squa...
2,Minute Player Squad x...


# Shots 